# Pure State Tomography Experiments
This notebook contains code for running experiments on IBM quantum computers.

## First Time Setup
Before running the code in this repository, you will need to set up the IBM Qiskit Runtime. First, create a file named config.ini, which will contain your IBM credentials. The format is as follows:

```ini
[IBM]
token = <YOUR_IBM_TOKEN>
```

This program will read the config.ini file for your API token. Now, run the following code to save your IBM token to your machine for further use. You will no longer need to run this cell afterwards. 

In [1]:
import configparser
from qiskit_ibm_runtime import QiskitRuntimeService

with open("config.ini", "r") as cf:
    cp = configparser.ConfigParser()
    cp.read_file(cf)
    api_token = cp.get("IBM", "token")
QiskitRuntimeService.save_account(channel="ibm_quantum", token=api_token, overwrite=True)

You will also need to install the prerequisite python packages.

### Running Experiments
First, import the necessary packages and set up the code:

In [2]:
from numpy import (
    ndarray,
    array,
    reshape,
    sqrt,
    linalg,
    set_printoptions
)
from pure_state_tomography import tomography
import putils
import qutils
from measurement_manager import measurement_manager

set_printoptions(linewidth=100)

The following function runs the experiment using the `pure_state_tomography` package. 

For runs on real IBM quantum computers (i.e. not simulator or statevector runs), the first concluded run for an input will output a job file containing the IBM job ids, which the `tomography` class will use to recall the contents of previous runs. Do note that because concurrent IBM jobs are limited to 3 per free account, you may have to run this code several times to get all of the needed measurement results.  

In [6]:
talg = tomography()

def run(
    mm: measurement_manager,
    state: ndarray,
    verbose: bool = True,
    job_file: str = None,
    hadamard: bool = True,
):
    putils.fprint("Running inference at {} shots\n".format(mm.n_shots))

    # reset measurement manager state
    mm.set_state(state)
    
    # run pure state tomography
    res = talg.pure_state_tomography(
        mm=mm, verbose=verbose, job_file=job_file, hadamard=hadamard
    )

    if res is not None:
        if state.ndim > 1:
            res = (
                reshape(
                    res,
                    (
                        state.shape[0],
                        state.shape[0],
                    ),
                ).T
                * 2
            )
        putils.fprint(
            "Reconstructed {}:\n{}".format(
                "vector" if state.ndim == 1 else "matrix", res
            )
        )
        putils.fprint(
            "% Error: {}\n".format(100 * linalg.norm(state- res))
        )

The following code runs the experiment. Using a single measurement manager object is recommended. 

In [9]:
VERBOSITY = True

state = array([1 / 2, 0, 0, -3 / sqrt(12)])

mm = measurement_manager(
    n_shots=16384,
    execution_type=qutils.execution_type.simulator,
    verbose=VERBOSITY
)

run(mm=mm, state=state, verbose=VERBOSITY, job_file=None, hadamard=False)

Running inference at 16384 shots

Input vector: [ 0.5        0.         0.        -0.8660254]
        ┌───────────────────────────────┐ ░  ░ ┌───┐ ░ ┌─┐   
   q_0: ┤0                              ├─░──░─┤ I ├─░─┤M├───
        │  Initialize(0.5,0,0,-0.86603) │ ░  ░ ├───┤ ░ └╥┘┌─┐
   q_1: ┤1                              ├─░──░─┤ I ├─░──╫─┤M├
        └───────────────────────────────┘ ░  ░ └───┘ ░  ║ └╥┘
meas: 2/════════════════════════════════════════════════╩══╩═
                                                        0  1 
Measurements:
Circuits for source index 0 and target index 2:
        ┌───────────────────────────────┐ ░  ░    ┌───┐    ░ ┌─┐   
   q_0: ┤0                              ├─░──░────┤ I ├────░─┤M├───
        │  Initialize(0.5,0,0,-0.86603) │ ░  ░ ┌──┴───┴──┐ ░ └╥┘┌─┐
   q_1: ┤1                              ├─░──░─┤ Unitary ├─░──╫─┤M├
        └───────────────────────────────┘ ░  ░ └─────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════════════════════════════════╩══╩═
  